In [2]:
import glob
import pandas as pd
import pickle
import cv2 
import numpy as np
import matplotlib.pyplot as plt

In [7]:
cov = glob.glob('D:/projects data/dataset/pnu/Covid19-dataset/train/Covid/*')
pne = glob.glob('D:/projects data/dataset/pnu/Covid19-dataset/train/Viral Pneumonia/*')
norm = glob.glob('D:/projects data/dataset/pnu/Covid19-dataset/train/Normal/*')



str

In [17]:
def preprocess(img):
    img= cv2.resize(img,(240,240))
    img=np.array(img)/255
    return img

In [11]:
len(cov),len(pne),len(norm)

(132, 70, 70)

In [12]:
132+70+70

272

In [18]:
x=[]
y=[]
for i in cov:
    im=cv2.imread(i)
    x.append(preprocess(im))
    y.append(0)
    
for i in pne:
    im=cv2.imread(i)
    x.append(preprocess(im))
    y.append(1)
    
for i in norm:
    im=cv2.imread(i)
    x.append(preprocess(im))
    y.append(2)

In [22]:
# im=cv2.imread(x[0])
x=np.array(x)
x.shape

(272, 240, 240, 3)

In [27]:
y=np.array(y)
y.shape

(272,)

In [33]:
#apply 1hot encoding on y
import sklearn.preprocessing
y=np.array(y)
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(y)+1))
y = label_binarizer.transform(y)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [28]:
import  pickle 
pickle.dump(x, open("pneu_data_0.pkl", 'wb'))
pickle.dump(y, open("pneu_labeles_0.pkl", 'wb'))

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.1)

In [36]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(3, activation='softmax')
])

In [40]:
model.compile(optimizer = 'adam', loss = tf.keras.losses.categorical_crossentropy, metrics = ['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
8/8 [==============================] - 8s 709ms/step - loss: 0.8291 - accuracy: 0.6557
Epoch 2/10
8/8 [==============================] - 3s 390ms/step - loss: 0.5554 - accuracy: 0.8279
Epoch 3/10
8/8 [==============================] - 4s 435ms/step - loss: 0.4161 - accuracy: 0.8811
Epoch 4/10
8/8 [==============================] - 5s 616ms/step - loss: 0.3358 - accuracy: 0.8934
Epoch 5/10
8/8 [==============================] - 6s 791ms/step - loss: 0.2931 - accuracy: 0.9098
Epoch 6/10
8/8 [==============================] - 4s 497ms/step - loss: 0.2615 - accuracy: 0.9262
Epoch 7/10
8/8 [==============================] - 1s 80ms/step - loss: 0.2365 - accuracy: 0.9385
Epoch 8/10
8/8 [==============================] - 1s 79ms/step - loss: 0.2162 - accuracy: 0.9426
Epoch 9/10
8/8 [==============================] - 1s 79ms/step - loss: 0.1980 - accuracy: 0.9508
Epoch 10/10
8/8 [==============================] - 1s 79ms/step - loss: 0.1862 - accuracy: 0.9467


In [41]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.3876 - accuracy: 0.8214


[0.38764843344688416, 0.8214285969734192]

In [42]:
model.save('pneu_model.HDF5')

INFO:tensorflow:Assets written to: pneu_model.HDF5\assets


INFO:tensorflow:Assets written to: pneu_model.HDF5\assets
